# Annotation

In this notebook, we'll explore how to annotate various data sets. Annotation broadly refers to a range of techniques spanning from anomaly/change point detection to segmentation. Such techniques are used in a diverse range of applications, for instance, the identification of extreme weather events, stock market crashes and fraudulent transactions. 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from sktime.annotation.datagen import piecewise_normal_multivariate
from sktime.utils.validation._dependencies import _check_soft_dependencies

np.random.seed(123)

_check_soft_dependencies("ipympl", severity="warning")

%matplotlib widget

## 1. Data Generation

First, we'll generate some multivariate data that exhibits mean shifts (with constant covariance)

In [ ]:
mv_mean = piecewise_normal_multivariate(
    means=[[0, 3], [5, 0], [10, 3]],
    lengths=[30, 50, 20],
    covariances=[
        [[0.5, 0.3], [0.3, 1.0]],
        [[0.5, 0.3], [0.3, 1.0]],
        [[0.5, 0.3], [0.3, 1.0]],
    ],
    random_state=279,
)

fig, ax = plt.subplots(figsize=(12, 4))
ax.plot(mv_mean)
ax.legend(["y_0", "y_1"])

plt.title("Mean-shifted multivariate timeseries")
plt.show()

In the data space, this can be represented as such

In [ ]:
fig, ax = plt.subplots(figsize=(4, 4))
ax.plot(mv_mean[:, 0], mv_mean[:, 1], "o")

plt.title("Mean-shifted multivariate data")
plt.xlabel("y_0")
plt.ylabel("y_1")
plt.show()

Secondly, we generate a multivariate timeseires by changing the covariance structure (from independent random variables, to highly negatively correlated, to highly correlated). This can be seen in the covariances matrices input below.

In [ ]:
mv_cov = piecewise_normal_multivariate(
    means=[[1.1, 3], [1, 2.9], [1, 3]],
    lengths=[40, 50, 60],
    covariances=[
        [[1.0, 0.003], [0.003, 1.0]],
        [[1.0, -0.999], [-0.999, 1.0]],
        [[1.0, 1.0], [1.0, 1.0]],
    ],
    random_state=2354,
)

fig, ax = plt.subplots(figsize=(12, 4))
ax.plot(mv_cov)

ax.legend(["y_0", "y_1"])

plt.title("Covariance-shifted multivariate timeseries")
plt.show()

Representing this timeseries with changing covariace in the data space allows us to uncover interesting characteristics; the 45 degree angle line comes about from the phase of high correlation and the 135 degree line comes about from the phase of negative correlation.

In [ ]:
fig, ax = plt.subplots(figsize=(4, 4))
ax.plot(mv_cov[:, 0], mv_cov[:, 1], "o")

plt.title("Covariance-shifted multivariate data")
plt.xlabel("y_0")
plt.ylabel("y_1")
plt.show()

Note the different definitions of "dimension": we use a 2-dimensional array to represent tabular data, but also describe the dimensionality of data (in the data space above, which is the number of columns). Intrinsic dimensionality is something else entirely...

## 2. Data Analysis

Suppose that we want to understand phases and/or changing complex system regimes (with potentially hidden states). We will now explore different techniques to segment the data.

### HMM: Hidden Markov Model

A hidden Markov model (HMM) allows us to account for both observed events and hidden events that we think of as causal factors in our probabilistic model. This is done by fitting a sequence of hidden states to a sequence of observations, by finding the most likely path given the emission probabilities - (i.e. the probability that a particular observation
would be generated by a given hidden state), the transition probability (i.e. the probability of transitioning from one state to another or staying in the same state) and the initial probabilities - i.e. the belief of the probability distribution of hidden states at the
start of the observation sequence).

In [ ]:
from sktime.annotation.hmm_learn import GMMHMM

X = mv_mean[:, 0]
model = GMMHMM(n_components=3, random_state=123)
model = model.fit(X)
labels = model.predict(X)

fig, axs = plt.subplots(2, 1, figsize=(12, 5), gridspec_kw={"height_ratios": [5, 1]})
axs[0].plot(X)
axs[1].plot(labels, "o")

fig.suptitle("HMM change point detection of univariate timeseries (mean-shifted)")
fig.tight_layout()
plt.show()

#### HIDALGO: Heterogeneous Intrinsic Dimensionality ALGOrithm

A fundamental paradigm in machine learning is that a small number of variables is often sufficient to describe high-dimensional data, where Intrinsic dimensionality (ID) is the minimum number of variables required. In most approaches for dimensionality reduction and manifold learning, the ID is assumed to be constant in the dataset. However, this assumption is often violated in the case of real-world data. Heterogeneous ID allows for data to be represented by a mixture model on the support on the union of K manifolds with varying dimensions. Points are then assigned to manifold k with corresponding dimensionality. Hidalgo extends this concept to a Bayesian framework which allows the identification (by Gibbs sampling) of regions in the data landscape where the ID can be considered constant. Hidalgo offers robust method for unsupervised segmentation of high-dimensional data [1].

In [ ]:
from sktime.transformations.series.hidalgo import Hidalgo

model = Hidalgo(K=3, burn_in=0.8, n_iter=100, seed=10)
fitted_model = model.fit(mv_mean)
z_mv_mean = fitted_model.transform(mv_mean)

fig, axs = plt.subplots(2, 1, figsize=(12, 5), gridspec_kw={"height_ratios": [5, 1]})
axs[0].plot(mv_mean)
axs[1].plot(z_mv_mean, "o")

fig.suptitle("HIDALGO segmentation of multivariate timeseries (mean-shifted)")
fig.tight_layout()
plt.show()

In [ ]:
model = Hidalgo(K=2, burn_in=0.8, n_iter=1000, seed=148)
fitted_model = model.fit(mv_cov)
z_mv_cov = fitted_model.transform(mv_cov)

fig, axs = plt.subplots(2, 1, figsize=(12, 5), gridspec_kw={"height_ratios": [5, 1]})
axs[0].plot(mv_cov)
axs[1].plot(z_mv_cov, "o")

fig.suptitle("HIDALGO segmentation of multivariate timeseries (covariance-shifted)")
fig.tight_layout()
plt.show()

#### STRAY: Search TRace AnomalY outlier estimator

STRAY [2] is an extension of HDoutliers [3], a powerful algorithm for the detection of anomalous observations in a dataset which has (among other advantages) the ability to detect clusters of outliers in multi-dimensional data without requiring a model of the typical behavior of the system. However, it suffers from some limitations that affect its accuracy. STRAY is an extension of HDoutliers that uses extreme value theory for the anomalous threshold calculation, to deal with data streams that exhibit non-stationary behavior.

In [ ]:
from sklearn.preprocessing import RobustScaler

from sktime.annotation.stray import STRAY

scaler = RobustScaler()
X = scaler.fit_transform(mv_mean)
model = STRAY(alpha=0.1)
y_actual = model.fit_transform(X)

fig, axs = plt.subplots(2, 1, figsize=(12, 5), gridspec_kw={"height_ratios": [5, 1]})
axs[0].plot(X)
axs[1].plot(y_actual, "o")

fig.suptitle("Anomaly detection for multivariate timeseries (mean-shifted)")
fig.tight_layout()
plt.show()

In [ ]:
scaler = RobustScaler()
X = scaler.fit_transform(mv_mean)
X[10][0] = 1.2
X[10][1] = -0.8

model = STRAY(alpha=0.1)
y_actual = model.fit_transform(X)

fig, axs = plt.subplots(2, 1, figsize=(12, 5), gridspec_kw={"height_ratios": [5, 1]})
axs[0].plot(X)
axs[1].plot(y_actual, "o")

fig.suptitle(
    "Anomaly detection for multivariate timeseries with outlier (mean-shifted)"
)
fig.tight_layout()
plt.show()

## 3. Building Pipelines

`sktime` has a range of pre-processing transformations available for use, to build analysis pipelines. There is a partcular method; a dimensionality reductuion technique specifically for the purpose of anomaly detection tasks (see below).

#### DOBIN: Distance based Outlier BasIs using Neighbors

A common challenge in many domains is that data is high-dimensional. Principal Component Analysis (PCA) is often used for dimension reduction when detecting outliers, however PCA finds a set of basis vectors that explains the variance of the data, such that the highest variance is in the direction of the first vector, and so on. But an outlier in the original high dimensional space might no longer be an outlier in the low dimesional subspace created by the dimension reduction method. So, outliers can be lost in this pre-processing step. DOBIN [4] constructs a set of basis vectors specifically tailored for unsupervised outlier detection to be used as a dimension reduction tool for outlier detection tasks.

In [ ]:
mv_mean_outlier = piecewise_normal_multivariate(
    means=[[-2, -2, 0], [2, 2, 0], [3, 3, 1], [0, -10, 0]],
    lengths=[30, 50, 20, 3],
    random_state=279,
)

fig, ax = plt.subplots(figsize=(12, 4))
ax.plot(mv_mean_outlier)

ax.legend(["y_0", "y_1", "y_2"])

plt.title("Mean-shifted multivariate timeseries")
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection="3d")
ax.plot(mv_mean_outlier[:, 0], mv_mean_outlier[:, 1], mv_mean_outlier[:, 2], "o")
ax.set_xlabel("y_0")
ax.set_ylabel("y_1")
ax.set_zlabel("y_2")

plt.title("Mean-shifted multivariate data")
plt.show()

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X = scaler.fit_transform(mv_mean_outlier)

pca = PCA(n_components=3)
X_pca = pca.fit_transform(X)

In [ ]:
fig, ax = plt.subplots(figsize=(4, 4))
ax.plot(X_pca[:, 0], X_pca[:, 1], "o")
ax.plot(X_pca[100:, 0], X_pca[100:, 1], "o", "r")

ax.set_xlabel("Princical Component 0")
ax.set_ylabel("Principal Component 1")
plt.show()

In [ ]:
import pandas as pd

from sktime.transformations.series.dobin import DOBIN

scaler = MinMaxScaler()
X = scaler.fit_transform(mv_mean_outlier)
model = DOBIN()
X_outlier = model.fit_transform(pd.DataFrame(X))

In [ ]:
fig, ax = plt.subplots(figsize=(4, 4))
ax.plot(X_outlier["DB0"], X_outlier["DB1"], "o")
ax.plot(X_outlier["DB0"][100:], X_outlier["DB1"][100:], "o", "r")

ax.set_xlabel("DOBIN BASIS 0")
ax.set_ylabel("DOBIN BASIS 1")
plt.show()


### References:
[1] Allegra, Michele, et al. "Data segmentation based on the local
    intrinsic dimension." Scientific reports 10.1 (2020): 1-12.
    https://www.nature.com/articles/s41598-020-72222-0

[2] Talagala, Priyanga Dilini, Rob J. Hyndman, and Kate Smith-Miles.
"Anomaly detection in high-dimensional data." Journal of Computational
and Graphical Statistics 30.2 (2021): 360-374.

[3] Wilkinson, Leland. "Visualizing big data outliers through
distributed aggregation." IEEE transactions on visualization and
computer graphics 24.1 (2017): 256-266.

[4] Kandanaarachchi, Sevvandi, and Rob J. Hyndman. "Dimension reduction
    for outlier detection using DOBIN." Journal of Computational and Graphical
    Statistics 30.1 (2021): 204-219.

---


### Credits:

sktime: https://github.com/sktime/sktime/blob/main/CONTRIBUTORS.md